In [15]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer


In [2]:
df = pd.read_csv("train.csv")

In [3]:
df.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [4]:
df.shape

(404290, 6)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 404290 entries, 0 to 404289
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   id            404290 non-null  int64 
 1   qid1          404290 non-null  int64 
 2   qid2          404290 non-null  int64 
 3   question1     404289 non-null  object
 4   question2     404288 non-null  object
 5   is_duplicate  404290 non-null  int64 
dtypes: int64(4), object(2)
memory usage: 18.5+ MB


In [6]:
df.isnull().sum()

id              0
qid1            0
qid2            0
question1       1
question2       2
is_duplicate    0
dtype: int64

In [7]:
df.duplicated().sum()

np.int64(0)

In [8]:
df.columns

Index(['id', 'qid1', 'qid2', 'question1', 'question2', 'is_duplicate'], dtype='object')

In [11]:
new_df = df.sample(30000)

,id,qid1,qid2,question1,question2,is_duplicate
265465,265465,355438,382439,can we open Startup while working full time in...,Is there any hassle to get interview calls fom...,0


In [12]:
ques_df = new_df[["question1","question2"]]

In [13]:
ques_df

,question1,question2
313723,How will Trump or Clinton presidency affect fo...,How might Trump affect the status of foreign s...
223779,Which is the most expensive fruit in world?,What are the most expensive dry fruits in the ...
234355,What is the history of INSEAD's language requi...,"If a person is born deaf, which language do th..."
385266,What are some CV worthy online Digital Marketi...,Which is the best digital marketing institute ...
393362,How can a substance have chiral carbons but no...,What are optically active substances?
...,...,...
266536,Where can I found modern colours and textures ...,Where can I found a leading manufacturer and s...
338526,Why do alot of Moroccans have Iranian names?,How does a fingerprint scanner work on a smart...
369138,Is Donald Trump correct that President Barack ...,Has Obama been a bad president? What reasons d...
148570,What are some good ways to explain to a teache...,How do I prove my teacher wrong?


In [19]:
question = list(ques_df["question1"]) + list(ques_df["question2"])
cv = CountVectorizer(max_features=3000)
tf_idf = TfidfVectorizer(max_features=3000)
q1_arr,q2_arr = np.vsplit(cv.fit_transform(question).toarray(),2)


In [21]:
q1_arr


array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], shape=(30000, 3000))

In [24]:
df1 = pd.DataFrame(q1_arr,index=ques_df.index)
df2 = pd.DataFrame(q2_arr,index=ques_df.index)
final_df = pd.concat([df1, df2], axis=1)
final_df.shape


(30000, 6000)

In [28]:
final_df["is_duplicate"] = new_df["is_duplicate"]

In [31]:
final_df.head()

,0,1,2,3,4,5,6,7,8,9,...,2991,2992,2993,2994,2995,2996,2997,2998,2999,is_duplicate
313723,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
223779,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
234355,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
385266,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
393362,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [34]:
x = final_df.iloc[:,0:-1]
y = final_df.iloc[:,-1]

In [36]:
from sklearn.model_selection import train_test_split
x_train,x_test , y_train, y_test = train_test_split(x,y,test_size=0.2,random_state=42)

In [37]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report,accuracy_score

RF = RandomForestClassifier()
GNB = GaussianNB()


In [38]:
RF.fit(x_train,y_train)
GNB.fit(x_train,y_train)

,"priors priors: array-like of shape (n_classes,), default=NonePrior probabilities of the classes. If specified, the priors are notadjusted according to the data.",None
,"var_smoothing var_smoothing: float, default=1e-9Portion of the largest variance of all features that is added tovariances for calculation stability... versionadded:: 0.20",1e-09


In [39]:
y_pred_rf = RF.predict(x_test)
y_pred_gnb =GNB.predict(x_test)
print("Random Forest:",accuracy_score(y_test,y_pred_rf))
print(" ") 
print("Random Forest:",accuracy_score(y_test,y_pred_gnb)) 

Random Forest: 0.7433333333333333
 
Random Forest: 0.564
